# Flood mapping using Sentinel 1

In [2]:
import ee 
import geemap

In [3]:
ee.Authenticate()
ee.Initialize()

### Change geometry and data to your area of interest

In [4]:
geometry = ee.Geometry.Polygon([
          [
            [
              -121.99195680157558,
              38.815390498416065
            ],
            [
              -121.99195680157558,
              35.74580921067127
            ],
            [
              -118.7863346754292,
              35.74580921067127
            ],
            [
              -118.7863346754292,
              38.815390498416065
            ],
            [
              -121.99195680157558,
              38.815390498416065
            ]
          ]
        ])

before_start= '2024-04-01'
before_end='2024-04-10'

after_start='2024-04-16'
after_end='2024-04-30'

### Parameter settings

* Polarization: 'VH' # or 'VV'
* Pass Direction: 'DESCENDING' # or 'ASCENDING'
* Difference Threshold: 1.25

In [5]:
polarization = "VH" #or 'VV' --> VH mostly is the prefered polarization for flood mapping.
pass_direction = "DESCENDING" # or 'ASCENDING' when images are being compared use only only one pass direction.
difference_threshold = 1.25 #threshodl to be applied on the difference image (after flood - before flood). It has been chosen by trial and error. In case your flood extent result shows many false-positive or negative signals, consider changing it!


### Flood mapping

Here we used the following inputs & settings to generate the flood map and the exposure maps:

1. Imagery used: Sentinel 1 (_COPERNICUS/S1_GRD_)
2. Masked the permanemtn water (where there is water > 10 months of the year) using [JRC layer](https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_4_GlobalSurfaceWater) on surface water seasonality (_JRC/GSW1_4/GlobalSurfaceWater_).
3. Maksed the more than 5 percentage slope using [WWF HydroSHEDS DEM](https://developers.google.com/earth-engine/datasets/catalog/WWF_HydroSHEDS_03VFDEM) (_WWF/HydroSHEDS/03VFDEM_).
4. Number of people exposure is calculated using [JRC Global Human Settlement Popluation Density layer](https://human-settlement.emergency.copernicus.eu/index.php) (_JRC/GHSL/P2016/POP_GPW_GLOBE_V1/2015_) - 250m resolution.
5. Cropland & Urban exposure is calculated using [MODIS Land Cover Type Yearly Global layer]() (_MODIS/006/MCD12Q1_) - 500m resolution.

In [9]:
m = geemap.Map()

# rename selected geometry feature
aoi = ee.FeatureCollection(geometry)

# Load and filter Sentinel-1 GRD data by predefined parameters
collection= ee.ImageCollection('COPERNICUS/S1_GRD') \
  .filter(ee.Filter.eq('instrumentMode','IW')) \
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
  .filter(ee.Filter.eq('orbitProperties_pass',pass_direction)) \
  .filter(ee.Filter.eq('resolution_meters',10)) \
  .filterBounds(aoi) \
  .select(polarization)

# Select images by predefined dates
before_collection = collection.filterDate(before_start, before_end)
after_collection = collection.filterDate(after_start,after_end)


def dates(imgcol):
  range = imgcol.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
  printed = ee.String('from ') \
    .cat(ee.Date(range.get('min')).format('YYYY-MM-dd')) \
    .cat(' to ') \
    .cat(ee.Date(range.get('max')).format('YYYY-MM-dd'))
  return printed

# print dates of before images to console
before_count = before_collection.size()
# print(ee.String('Tiles selected: Before Flood ').cat('(').cat(before_count).cat(')'),
#   dates(before_collection), before_collection)

# print dates of after images to console
after_count = after_collection.size()
# print(ee.String('Tiles selected: After Flood ').cat('(').cat(after_count).cat(')'),
#   dates(after_collection), after_collection)

# Create a mosaic of selected tiles and clip to study area
before = before_collection.mosaic().clip(aoi)
after = after_collection.mosaic().clip(aoi)

# Apply reduce the radar speckle by smoothing
smoothing_radius = 50
before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters')
after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters')

#------------------------------- FLOOD EXTENT CALCULATION -------------------------------#

# Calculate the difference between the before and after images
difference = after_filtered.divide(before_filtered)

# Apply the predefined difference-threshold and create the flood extent mask
threshold = difference_threshold
difference_binary = difference.gt(threshold)

# Refine flood result using additional datasets

# Include JRC layer on surface water seasonality to mask flood pixels from areas
# of "permanent" water (where there is water > 10 months of the year)
swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality')
swater_mask = swater.gte(10).updateMask(swater.gte(10))

#Flooded layer where perennial water bodies (water > 10 mo/yr) is assigned a 0 value
flooded_mask = difference_binary.where(swater_mask,0)
# final flooded area without pixels in perennial waterbodies
flooded = flooded_mask.updateMask(flooded_mask)

# Compute connectivity of pixels to eliminate those connected to 8 or fewer neighbours
# This operation reduces noise of the flood extent product
connections = flooded.connectedPixelCount()
flooded = flooded.updateMask(connections.gte(8))

# Mask out areas with more than 5 percent slope using a Digital Elevation Model
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM')
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope')
flooded = flooded.updateMask(slope.lt(5))

# Calculate flood extent area
# Create a raster layer containing the area information of each pixel
flood_pixelarea = flooded.select(polarization) \
  .multiply(ee.Image.pixelArea())

# Sum the areas of flooded pixels
# default is set to 'bestEffort: True' in order to reduce compuation time, for a more
# accurate result set bestEffort to False and increase 'maxPixels'.
flood_stats = flood_pixelarea.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': aoi,
  'scale': 10, # native resolution
  #maxPixels: 1e9,
  'bestEffort': True
  })

# Convert the flood extent to hectares (area calculations are originally given in meters)
flood_area_ha = flood_stats \
  .getNumber(polarization) \
  .divide(10000) \
  .round()

print("Estimated flooded area (ha): ", flood_area_ha.getInfo())

#------------------------------  DAMAGE ASSSESSMENT  ----------------------------------#

#----------------------------- Exposed population density ----------------------------#

# Load JRC Global Human Settlement Popluation Density layer
# Resolution: 250. Number of people per cell is given.
population_count = ee.Image('JRC/GHSL/P2016/POP_GPW_GLOBE_V1/2015').clip(aoi)

# Calculate the amount of exposed population
# get GHSL projection
GHSLprojection = population_count.projection()

# Reproject flood layer to GHSL scale
flooded_res1 = flooded \
    .reproject(**{
    'crs': GHSLprojection
  })

# Create a raster showing exposed population only using the resampled flood layer
population_exposed = population_count \
  .updateMask(flooded_res1) \
  .updateMask(population_count)

#Sum pixel values of exposed population raster
stats = population_exposed.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': aoi,
  'scale': 250,
  'maxPixels':1e9
})

# get number of exposed people as integer
number_pp_exposed = stats.getNumber('population_count').round()

print("Estimated exposed number of people: ", number_pp_exposed.getInfo())

#----------------------------- Affected agricultural land ----------------------------#

# using MODIS Land Cover Type Yearly Global 500m
# filter image collection by the most up-to-date MODIS Land Cover product
LC = ee.ImageCollection('MODIS/006/MCD12Q1') \
  .filterDate('2014-01-01',after_end) \
  .sort('system:index',False) \
  .select("LC_Type1") \
  .first() \
  .clip(aoi)

# Extract only cropland pixels using the classes cropland (>60%) and Cropland/Natural
# Vegetation Mosaics: mosaics of small-scale cultivation 40-60% incl. natural vegetation
cropmask = LC \
  .eq(12) \
  .Or(LC.eq(14))
cropland = LC \
  .updateMask(cropmask)

# get MODIS projection
MODISprojection = LC.projection()

# Reproject flood layer to MODIS scale
flooded_res = flooded \
    .reproject(**{
    'crs': MODISprojection
  })

# Calculate affected cropland using the resampled flood layer
cropland_affected = flooded_res \
  .updateMask(cropland)

# get pixel area of affected cropland layer
crop_pixelarea = cropland_affected \
  .multiply(ee.Image.pixelArea()); 

# sum pixels of affected cropland layer
crop_stats = crop_pixelarea.reduceRegion(**{
  'reducer': ee.Reducer.sum(), #sum all pixels with area information
  'geometry': aoi,
  'scale': 500,
  'maxPixels': 1e9
  })

# convert area to hectares
crop_area_ha = crop_stats \
  .getNumber(polarization) \
  .divide(10000) \
  .round()

print("Estimated crop area affected (ha): ", crop_area_ha.getInfo())

#-------------------------------- Affected urban area ------------------------------#

# Using the same MODIS Land Cover Product
# Filter urban areas
urbanmask = LC.eq(13)
urban = LC \
  .updateMask(urbanmask)

#Calculate affected urban areas using the resampled flood layer
urban_affected = urban \
  .mask(flooded_res) \
  .updateMask(urban)

# get pixel area of affected urban layer
urban_pixelarea = urban_affected \
  .multiply(ee.Image.pixelArea()); 

# sum pixels of affected cropland layer
urban_stats = urban_pixelarea.reduceRegion(**{
  'reducer': ee.Reducer.sum(), #sum all pixels with area information
  'geometry': aoi,
  'scale': 500,
  'bestEffort': True,
  })

# convert area to hectares
urban_area_ha = urban_stats \
  .getNumber('LC_Type1') \
  .divide(10000) \
  .round()

print("Estimated urban area affected (ha): ", urban_area_ha.getInfo())

#------------------------------  DISPLAY PRODUCTS  ----------------------------------#

# Before and after flood SAR mosaic
m.centerObject(aoi,8)
m.addLayer(before_filtered, {'min':-25, 'max':0}, 'Before Flood',0)
m.addLayer(after_filtered, {'min':-25, 'max':0}, 'After Flood',1)

# Difference layer
m.addLayer(difference,{'min':0, 'max':2},"Difference Layer",0)

# Flooded areas
m.addLayer(flooded,{'palette':"0000FF"},'Flooded areas')

# Population Density
populationCountVis = {
  'min': 0,
  'max': 200.0,
  'palette': ['060606','337663','337663','ffffff'],
}
m.addLayer(population_count, populationCountVis, 'Population Density',0)

# Exposed Population
populationExposedVis = {
  'min': 0,
  'max': 200.0,
  'palette': ['yellow', 'orange', 'red'],
}
m.addLayer(population_exposed, populationExposedVis, 'Exposed Population')

# MODIS Land Cover
LCVis = {
  'min': 1.0,
  'max': 17.0,
  'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
}
m.addLayer(LC, LCVis, 'Land Cover',0)

# Cropland
croplandVis = {
  'min': 0,
  'max': 14.0,
  'palette': ['30b21c'],
}
m.addLayer(cropland, croplandVis, 'Cropland',0)

# Affected cropland
m.addLayer(cropland_affected, croplandVis, 'Affected Cropland')

# Urban
urbanVis = {
  'min': 0,
  'max': 13.0,
  'palette': ['grey'],
}
m.addLayer(urban, urbanVis, 'Urban',0)

# Affected urban
m.addLayer(urban_affected, urbanVis, 'Affected Urban')

Estimated flooded area (ha):  290933
Estimated exposed number of people:  6951
Estimated crop area affected (ha):  87193
Estimated urban area affected (ha):  7802


In [8]:
m

Map(center=[37.28095837641983, -120.3891457385023], controls=(WidgetControl(options=['position', 'transparent_…